# Train word2vec

In [2]:
import pickle
import numpy as np

from src.knowledge_db import TOKENS

In [3]:
# with open("/export/share/krausef99dm/data/data_test/test_9.0k_data.pkl", 'rb') as f:
with open(r"C:\Users\Felix\code\uni\UniVie\master-thesis\data\train_data\dev_train_9.0k_data.pkl", 'rb') as f:
    rna_data, target_ids, targets, targets_bin = pickle.load(f)

In [4]:
rna_data[0].shape

torch.Size([3382, 4])

In [36]:
rna_data[0].shape

torch.Size([3382])

## Sequence only

In [5]:
rna_data[0][:, 0].unique()

tensor([6, 7, 8, 9], dtype=torch.int8)

In [6]:
# remove all columns except the sequence
rna_data = [rna_data[i][:, 0] for i in range(len(rna_data))]

In [7]:
# build dictionary to map integers to tokens
int2token = {i+1: token for i, token in enumerate(TOKENS)}

In [8]:
def generate_kmers(rna_data, k=3):
    rna_data_kmers = []
    for seq in rna_data:
        seq = seq.tolist()
        seq = [int2token.get(i) for i in seq]
        seq_kmers = ["".join(seq[i:i+k]) for i in range(len(seq) - k + 1)]
        rna_data_kmers.append(seq_kmers)
        
    return rna_data_kmers

In [9]:
seq_kmers = generate_kmers(rna_data)

In [33]:
import gensim

embedding_size = 64

model = gensim.models.Word2Vec(seq_kmers, vector_size=embedding_size, window=12, min_count=0, workers=4)

In [59]:
import torch
import torch.nn as nn

# Get vocabulary and embeddings
vocab = list(model.wv.index_to_key)
embedding_matrix = torch.tensor(np.array([model.wv[word] for word in vocab], dtype=np.float32))

# Create k-mer to index mapping
kmer_to_index = {kmer: idx for idx, kmer in enumerate(vocab)}

In [62]:
# Store model
model_data = {"kmer_to_index": kmer_to_index, "embedding_matrix": embedding_matrix}
with open("../data/w2v_model_data.pkl", 'wb') as f:
    pickle.dump(model_data, f)

In [ ]:
# Load model
with open("../data/w2v_model_data.pkl", 'rb') as f:
    model_data = pickle.load(f)

In [58]:
class KMerEmbedding(nn.Module):
    def __init__(self, embedding_matrix):
        super().__init__()
        num_kmers, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(num_kmers, embedding_dim)
        self.embedding.weight.data.copy_(embedding_matrix)
        self.embedding.weight.requires_grad = False  # Freeze embeddings

    def forward(self, kmer_indices):
        return self.embedding(kmer_indices)

In [47]:
kmer_embedding = KMerEmbedding(embedding_matrix)

In [50]:
# Test embedding
kmer_indices = torch.tensor([kmer_to_index[kmer] for kmer in seq_kmers[0]], dtype=torch.long)
kmer_embedding(kmer_indices)

tensor([[ 0.7513,  0.2407, -1.6712,  ...,  0.4651, -0.4699, -0.2740],
        [-0.0577, -0.8098,  1.9476,  ...,  1.3473,  0.2246,  0.7454],
        [ 0.5385, -2.2471, -0.7330,  ..., -3.2796, -1.5477,  0.4677],
        ...,
        [-0.9429, -0.8822,  0.0987,  ..., -1.2557, -1.4814,  2.9707],
        [-1.1262, -1.8452, -1.2386,  ...,  2.5371, -2.4415, -1.7634],
        [-0.7141, -0.5241,  0.8825,  ...,  2.6616, -0.5948,  0.2421]])

## Embeddings for all features